# Text Classification Demo





# Introduction

In this demo, we will show how to build a production-ready CNN model for text classification using the DeText framework. The task is for query intent classification.

## Tutorial setup


### Outline
* Environment setup
* Data preprocessing
* Model training
* Inference with trained model


###   Dataset 
We will use the publicly available Natural Language Understanding benchmark dataset ([nlu-benchmark](https://github.com/snipsco/nlu-benchmark/tree/master/2017-06-custom-intent-engines)). The labeled data contains 7 intents including the following: 
* SearchCreativeWork (e.g. Find me the I, Robot television show),
* GetWeather (e.g. Is it windy in Boston, MA right now?),
* BookRestaurant (e.g. I want to book a highly rated restaurant for me and my boyfriend tomorrow night),
* ~~PlayMusic (e.g. Play the last track from Beyoncé off Spotify),~~ This is not included in this tutorial due to dataset download issues.
* AddToPlaylist (e.g. Add Diamonds to my roadtrip playlist)
* RateBook (e.g. Give 6 stars to Of Mice and Men)
* SearchScreeningEvent (e.g. Check the showtimes for Wonder Woman in Paris)

We will use the 6 valid classes to train the query intent classifier.

### Model 
Users will see how to train a CNN model for text classification with DeText. The trained model is ready for online inference in production search systems.


### Time required
appox. 20min



# Set up the environment
We first need to install detext using pip.

In [ ]:
!pip install detext==2.0.8


     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 17.4MB 29.9MB/s 
     |████████████████████████████████| 109.2MB 78kB/s 
     |████████████████████████████████| 40kB 5.3MB/s 
     |████████████████████████████████| 491kB 40.9MB/s 
     |████████████████████████████████| 3.2MB 37.8MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for detext: filename=detext-2.0.8-cp36-none-any.whl size=77060 sha256=631fe1dbff9c62177e7bfec789d022f05c6f482a4a6252240d6eee5337aa70f4
  Stored in directory: /root/.cache/pip/wheels/3d/3e/1e/f093368c4b09221180f4e804ff4d766b9f1015f807613c5520
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=8cf982571b2bb59e726007ddf5ff5aaa84ef6c9011f09db1f3b97621129c5d5a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built detext gast
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have nu

# Download and preprocess the dataset
In this section, we download the dataset and perform some basic preprocessing to the text data, including lowercasing, whitespace normalization, etc.



In [ ]:
!git clone https://github.com/snipsco/nlu-benchmark.git

Cloning into 'nlu-benchmark'...
remote: Enumerating objects: 389, done.
remote: Total 389 (delta 0), reused 0 (delta 0), pack-reused 389
Receiving objects: 100% (389/389), 1.19 MiB | 10.21 MiB/s, done.
Resolving deltas: 100% (246/246), done.


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [ ]:
# The constants
CLASSES = [
           "AddToPlaylist", 
           "BookRestaurant", 
           "GetWeather", 
           "RateBook", 
           "SearchCreativeWork", 
           "SearchScreeningEvent"
           ]
# See https://github.com/linkedin/detext/blob/master/TRAINING.md for more details on the feature naming format required by DeText
QUERY_FIELDNAME = "doc_query"
WIDE_FTR_FIELDNAME = "wide_ftrs"
LABEL_FIELDNAME = "label"
VOCAB_FILE = "vocab.txt"

In [ ]:
import json


train_raw = {}
test_raw = {}

for c in CLASSES:
  with open('/content/nlu-benchmark/2017-06-custom-intent-engines/{}/train_{}_full.json'.format(c, c)) as f:
    train_raw[c] = json.load(f)[c]
  
  with open('/content/nlu-benchmark/2017-06-custom-intent-engines/{}/validate_{}.json'.format(c, c)) as f:
    test_raw[c] = json.load(f)[c]
  
  print("Training samples for {}: {}".format(c, len(train_raw[c])), ", test samples for {}: {}".format(c, len(test_raw[c])))


Training samples for AddToPlaylist: 1942 , test samples for AddToPlaylist: 100
Training samples for BookRestaurant: 1973 , test samples for BookRestaurant: 100
Training samples for GetWeather: 2000 , test samples for GetWeather: 100
Training samples for RateBook: 1956 , test samples for RateBook: 100
Training samples for SearchCreativeWork: 1954 , test samples for SearchCreativeWork: 100
Training samples for SearchScreeningEvent: 1959 , test samples for SearchScreeningEvent: 100


In [ ]:
import re
import string

def proprocess_data(data_raw):
  X = []
  y = []
  for c in CLASSES:
    for sample in data_raw[c]:
      tokens = []
      for d in sample['data']:
        t = d['text'].strip().lower()
        t = re.sub(r'([%s])' % re.escape(string.punctuation), r' \1 ', t) 
        t = re.sub(r'\\.', r' ', t) 
        t = re.sub(r'\s+', r' ', t) 
        tokens.append(t)
      sentence = ' '.join(tokens)

      X.append(sentence)
      y.append(CLASSES.index(c))
  return X, y

    
X_train, y_train = proprocess_data(train_raw)
X_test, y_test = proprocess_data(test_raw)

print(X_train[:5])
print(y_train[:5])

['add another song to the cita romántica playlist . ', 'add clem burke in my playlist pre - party r & b jams', 'add live from aragon ballroom to trapeo', 'add unite and win to my  night out', 'add track to my  digster future hits']
[0, 0, 0, 0, 0]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [ ]:
# Stats
print('Total train samples: {}'.format(len(y_train)))
print('Total dev samples: {}'.format(len(y_dev)))
print('Total test samples: {}'.format(len(y_test)))

Total train samples: 10605
Total dev samples: 1179
Total test samples: 600


# Prepare DeText dataset

Now that we have preprocessed the datasets and prepared the train/ dev/ test splits, let's convert the data samples into the correct format that DeText accepts. 

The text inputs are converted to `byte_list` features and labels are converted to `float_list` features. The datasets are store in `tfrecord` format. For more information on the input format of DeText, please checkout [the training manual](https://github.com/linkedin/detext/blob/master/TRAINING.md).

In [ ]:
import tensorflow as tf
 
def create_float_feature(value): 
    feature = tf.train.Feature(float_list=tf.train.FloatList(value=[value])) 
    return feature 
 
 
def create_byte_feature(value): 
    feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    return feature 
 

def create_tfrecords(X, y, output_file):
  assert len(X) == len(y)
  total_records = 0 
  writer = tf.python_io.TFRecordWriter(output_file) 
  for q, c in zip(X, y):
    features = {} 
    features[QUERY_FIELDNAME] = create_byte_feature(str.encode(q))
    features[LABEL_FIELDNAME] = create_float_feature(c)
    features[WIDE_FTR_FIELDNAME] = create_float_feature(len(q.split()))
    tf_example = tf.train.Example(features=tf.train.Features(feature=features)) 
    if total_records < 2: 
      print(tf_example) 
    writer.write(tf_example.SerializeToString()) 
    total_records += 1 
  print("processed {} records".format(total_records)) 

create_tfrecords(X_train, y_train, 'train.tfrecord')
create_tfrecords(X_dev, y_dev, 'dev.tfrecord')
create_tfrecords(X_test, y_test, 'test.tfrecord')

features {
  feature {
    key: "doc_query"
    value {
      bytes_list {
        value: "i want to bring antoinette , christine and caitlin to get bread at a restaurant in faraway  connecticut"
      }
    }
  }
  feature {
    key: "label"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "wide_ftrs"
    value {
      float_list {
        value: 18.0
      }
    }
  }
}

features {
  feature {
    key: "doc_query"
    value {
      bytes_list {
        value: "book a reservation for 4 at a bar serving corn relish"
      }
    }
  }
  feature {
    key: "label"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "wide_ftrs"
    value {
      float_list {
        value: 11.0
      }
    }
  }
}

processed 10605 records
features {
  feature {
    key: "doc_query"
    value {
      bytes_list {
        value: "book a restaurant in everson  nevada that serves conchiglie for two people . "
      }
    }
  }
  feat

In [1]:
# Generate vocab
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)
vocab = ['[PAD]', '[UNK]']
vocab.extend(list(vectorizer.vocabulary_.keys()))

# Write vocab to file
with open(VOCAB_FILE, 'w') as f:
  for v in vocab:
    f.write("{}\n".format(v))


NameError: ignored

# DeText training
We will train a multi-class classification model using the DeText framework. 



In [ ]:
# See https://github.com/linkedin/detext/blob/master/TRAINING.md for more details on the parameters

from detext.run_detext import run_detext,DetextArg

args = DetextArg(num_classes=6,
                ftr_ext="cnn",
                num_filters=50,
                num_units=64,
                num_wide=1,
                optimizer="bert_adam", # same AdamWeightDecay optimizer as in BERT training
                learning_rate=0.001,
                max_len=16,
                min_len=3,
                use_deep=True,
                num_train_steps=300,
                steps_per_stats=1,
                steps_per_eval=30,
                train_batch_size=64,
                test_batch_size=64,
                pmetric="accuracy",
                all_metrics=["accuracy", "confusion_matrix"],
                vocab_file=VOCAB_FILE,
                feature_names=["label","doc_query","wide_ftrs"],
                train_file="train.tfrecord",
                dev_file="dev.tfrecord",
                test_file="test.tfrecord",
                out_dir="output")
run_detext(args)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





Unable to check if ['accuracy', 'confusion_matrix'] is of type typing.Union[typing.List[str], NoneType] for field all_metrics of argument class <class 'detext.run_detext.DetextArg'>
Unable to check if None is of type typing.Union[typing.List[float], NoneType] for field score_rescale of argument class <class 'detext.run_detext.DetextArg'>
Unable to check if None is of type typing.Union[typing.Dict[str, int], NoneType] for field hvd_info of argument class <class 'detext.run_detext.DetextArg'>
Unable to check if None is of type typing.Union[typing.List[int], NoneType] for field task_ids of argument class <class 'detext.run_detext.DetextArg'>
Unable to check if None is of type typing.Union[typing.List[float], NoneType] for field task_weights of argument class <class 'detext.run_detext.DetextArg'>


INFO:tensorflow:Getting executor context info...


INFO:tensorflow:Getting executor context info...


  saving hparams to output/hparams



...

INFO:tensorflow:metric/accuracy = 0.98833334


INFO:tensorflow:metric/accuracy = 0.98833334


INFO:tensorflow:metric/confusion_matrix = [[100   0   0   0   0   0]
 [  0 100   0   0   0   0]
 [  0   0  99   0   0   1]
 [  0   0   0  99   1   0]
 [  0   0   0   0 100   0]
 [  0   0   1   0   4  95]]


INFO:tensorflow:metric/confusion_matrix = [[100   0   0   0   0   0]
 [  0 100   0   0   0   0]
 [  0   0  99   0   0   1]
 [  0   0   0  99   1   0]
 [  0   0   0   0 100   0]
 [  0   0   1   0   4  95]]


INFO:tensorflow:output/export/best_accuracy/1598001178


# Let's check the predictions

In [ ]:
import glob
from tensorflow.contrib import predictor 
saved_model_path = glob.glob('output/export/best_accuracy/*')[0]

print(saved_model_path)

INFO:tensorflow:output/export/best_accuracy/1598001178


In [ ]:
!saved_model_cli show --dir output/export/best_accuracy/* --tag_set serve --signature_def serving_default

INFO:tensorflow:The given SavedModel SignatureDef contains the following input(s):
  inputs['doc_query'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: doc_query_placeholder:0
  inputs['label'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: label_placeholder:0
  inputs['uid'] tensor_info:
      dtype: DT_INT64
      shape: ()
      name: uid_placeholder:0
  inputs['weight'] tensor_info:
      dtype: DT_FLOAT
      shape: ()
      name: weight_placeholder:0
  inputs['wide_ftrs'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: wide_ftr_placeholder:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['label'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, -1)
      name: ExpandDims_2:0
  outputs['multiclass_probabilities'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 6)
      name: Softmax:0
  outputs['scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 6)
      name: Squeeze:0
  out

In [ ]:
import numpy as np
predict_fn = predictor.from_saved_model(saved_model_path) 

def predict_class(query):
  predictions = predict_fn( 
                  {"doc_query": [query], 
                   "wide_ftrs": [[len(query.split())]],
                   # dummy pass-through input for label field -- this is not used in making predictions
                   "label": [1.0]}) 
  probabilities = predictions['multiclass_probabilities'][0]
  predicted_class = CLASSES[np.argmax(probabilities)]
  print("query \"{}\", predicted intent \"{}\".".format(query, predicted_class))

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/export/best_accuracy/1598001178/variables/variables


In [ ]:
# GetWeather intent
predict_class("how's the weather in Mountain View")

# BookRestaurant intent
predict_class("I need a table for 4")

# PlayMusic intent
predict_class("put Jean Philippe Goncalves onto my running to rock 170 to 190 bpm")

# SearchScreeningEvent intent
predict_class("Find the schedule for The Comedian")

INFO:tensorflow:query "how's the weather in Mountain View", predicted intent "GetWeather".
INFO:tensorflow:query "I need a table for 4", predicted intent "BookRestaurant".
INFO:tensorflow:query "put Jean Philippe Goncalves onto my running to rock 170 to 190 bpm", predicted intent "AddToPlaylist".
INFO:tensorflow:query "Find the schedule for The Comedian", predicted intent "SearchScreeningEvent".


# References

\[1\] Dataset: Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces, [github repo](https://github.com/snipsco/nlu-benchmark)

